# Анализ лояльности пользователей Яндекс Афиши

In [ ]:
import pandas as pd
from pandas import DataFrame
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import phik
import os

In [ ]:
db_config = {'user': os.getenv('DB_USER'), # имя пользователя
             'pwd': os.getenv('DB_PASSWORD'), # пароль
             'host': os.getenv('DB_HOST'),
             'port': os.getenv('DB_PORT'), # порт подключения
             'db': os.getenv('DB_NAME') # название базы данных
             }

In [ ]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

In [ ]:
engine = create_engine(connection_string)

In [ ]:
def query_generator(table_name):
    return f'SELECT * FROM afisha.{table_name}'

def get_df(table_name):
    
    query = query_generator(table_name)
    
    return pd.read_sql_query(query, con=engine)

def collect_db_data_to_dataframes(schema_name) -> dict[str, DataFrame]:
    
    tables_query = f'''
    select table_name from information_schema.tables
    where table_schema = '{schema_name}'
    '''
        
    table_names = []
    with engine.connect() as conn:
        result = conn.execute(tables_query)
        for row in result:
            table_names.append(row[0])
    
    dfs = dict()
    for table_name in table_names:
        dfs[table_name] = get_df(table_name)
        
    return dfs

In [ ]:
# Содержит сырые данные по таблицам
dataframes = collect_db_data_to_dataframes('afisha')

In [ ]:
for df in dataframes.values():
    df.info()
    print('\n\n\n')

In [ ]:
interested_dataframe_query = '''
select
    user_id,
    device_type_canonical,
    order_id,
    created_dt_msk as order_dt,
    created_ts_msk as order_ts,
    currency_code,
    revenue,
    tickets_count,
    (extract(day from created_dt_msk - lag(created_dt_msk) over (partition by user_id order by created_dt_msk asc)))::int as days_since_prev,
    event_id,
    service_name,
    event_type_main,
    region_name,
    city_name
from afisha.purchases p
    join afisha.events e using(event_id)
    join afisha.city c using(city_id)
    join afisha.regions r using(region_id)
'''

interested_dataframe = pd.read_sql_query(interested_dataframe_query, con=engine)
idf = interested_dataframe

---

**Задача 1.2:** Изучите общую информацию о выгруженных данных. Оцените корректность выгрузки и объём полученных данных.

Предположите, какие шаги необходимо сделать на стадии предобработки данных — например, скорректировать типы данных.

Зафиксируйте основную информацию о данных в кратком промежуточном выводе.

---

<b>Вывод первых десяти строк датасета</b>

In [ ]:
idf.head(10)

<b>Вывод размерности датасета</b>

In [ ]:
idf.shape

<b>Вывод колонок с пропусками</b>

In [ ]:
idf.isna().sum()

<b>Вывод информации о типах данных колонок</b>

In [ ]:
idf.info()

<b>Вывод информации о количестве уникальных значений</b>

In [ ]:
idf.nunique()

<p>Резюмируем предварительный анализ данных.</p>
<ul>
    <li>Размер составляет 292034 строк и 14 столбцов</li>
    <li>Пропуски присутствуют только в колонке days_since_prev. Отсутствие пропуска в купе с количеством уникальных пользователей означает, что это первый заказ пользователя. </li>
    <li>Типы данных корректны во всех колонках. Исключение составляет та же колонка days_since_prev, из-за наличия NaN значений</li>
    <li>Количество заказов 292034</li>
    <li>Количество пользователей 22000</li>
    <li>Количество валют 2</li>
</ul>
<p>Из полученой информации, о данных можно сделать выводы о отсутствии необходимости корректировок чего либо.</p>

---

###  2. Предобработка данных

Выполните все стандартные действия по предобработке данных:

---

**Задача 2.1:** Данные о выручке сервиса представлены в российских рублях и казахстанских тенге. Приведите выручку к единой валюте — российскому рублю.

Для этого используйте датасет с информацией о курсе казахстанского тенге по отношению к российскому рублю за 2024 год — `final_tickets_tenge_df.csv`. Его можно загрузить по пути `https://code.s3.yandex.net/datasets/final_tickets_tenge_df.csv')`

Значения в рублях представлено для 100 тенге.

Результаты преобразования сохраните в новый столбец `revenue_rub`.

---


In [ ]:
currency_df = pd.read_csv('https://code.s3.yandex.net/datasets/final_tickets_tenge_df.csv')

In [ ]:
currency_df.info()

<b>Необходима корректировка типа данных у колонки date на datetime64[ns]. Это позволит нам выпонить merge датасетов</b>

In [ ]:
currency_df['data'] = pd.to_datetime(currency_df['data'])

In [ ]:
currency_df.info()

In [ ]:
currency_df.head(5)

In [ ]:
currency_adjusted_df = pd.merge(idf, currency_df, left_on='order_dt', right_on='data')

def adjust_currency(row):
    revenue = row['revenue']
    
    if row['currency_code'] == 'rub':
        return revenue
    
    return (revenue / row['nominal']) * row['curs']

currency_adjusted_df['revenue_rub'] = currency_adjusted_df.apply(adjust_currency, axis=1)

currency_adjusted_df = currency_adjusted_df.drop(columns=list(currency_df.columns))

display(currency_adjusted_df[currency_adjusted_df['currency_code'] == 'kzt'].head(5))

cadf = currency_adjusted_df

---

**Задача 2.2:**

- Проверьте данные на пропущенные значения. Если выгрузка из SQL была успешной, то пропуски должны быть только в столбце `days_since_prev`.
- Преобразуйте типы данных в некоторых столбцах, если это необходимо. Обратите внимание на данные с датой и временем, а также на числовые данные, размерность которых можно сократить.
- Изучите значения в ключевых столбцах. Обработайте ошибки, если обнаружите их.
    - Проверьте, какие категории указаны в столбцах с номинальными данными. Есть ли среди категорий такие, что обозначают пропуски в данных или отсутствие информации? Проведите нормализацию данных, если это необходимо.
    - Проверьте распределение численных данных и наличие в них выбросов. Для этого используйте статистические показатели, гистограммы распределения значений или диаграммы размаха.
        
        Важные показатели в рамках поставленной задачи — это выручка с заказа (`revenue_rub`) и количество билетов в заказе (`tickets_count`), поэтому в первую очередь проверьте данные в этих столбцах.
        
        Если обнаружите выбросы в поле `revenue_rub`, то отфильтруйте значения по 99 перцентилю.

После предобработки проверьте, были ли отфильтрованы данные. Если были, то оцените, в каком объёме. Сформулируйте промежуточный вывод, зафиксировав основные действия и описания новых столбцов.

---

<b>Проверка данных на пропущенные значений</b>

In [ ]:
cadf.isna().sum()

<b>Проверка на некорректные значения</b>

In [ ]:
cadf.aggregate(
    {
        'revenue': ['min', 'max'],
        'revenue_rub': ['min', 'max'],
        'days_since_prev': ['min', 'max'],
        'order_ts': ['min', 'max'],
        'order_ts': ['min', 'max'],
        
    }
)

In [ ]:
number_of_negative_revenue_rows = cadf[cadf['revenue_rub'] < 0]['revenue_rub'].count()

In [ ]:
f'Количество строк с отрицательным значением прибыли: {number_of_negative_revenue_rows}'

<b>Получаем 381 строку где прибль отрпцательна. Выполним удаление этих строк, т.е. по сравнению с общим количеством строк это значение значительно мало.</b>

In [ ]:
cadf = cadf[cadf['revenue_rub'] > 0].copy()
cadf[cadf['revenue_rub'] < 0]['revenue_rub'].count()

In [ ]:
cadf.info()

<b>Выполним преобразование типов данных</b>

In [ ]:
cadf['tickets_count'] = pd.to_numeric(cadf['tickets_count'], downcast='integer')
cadf['revenue_rub'] = pd.to_numeric(cadf['revenue_rub'], downcast='float')
cadf['days_since_prev'] = pd.to_numeric(cadf['days_since_prev'], downcast='float')

In [ ]:
cadf.info()

<b>Проверка колонок с номинальными данными на их уникальность с целью поиска значений обозначающих пропуски</b>

In [ ]:
cadf['device_type_canonical'].unique()


In [ ]:
cadf['currency_code'].unique()

In [ ]:
cadf['service_name'].unique()

In [ ]:
cadf['event_type_main'].unique()

In [ ]:
cadf['region_name'].unique()

In [ ]:
cadf['city_name'].unique()

<b>В колонках device_type_canonical и event_type_main присутствуют данные отражающие пропуски, а именно "other" и "другое" соответственно. Нормализацию данных проводить не требуется.</b>

<b>Анализ числовых данных на наличие выбросов</b>

In [ ]:
# удаляем колонки с идентификаторами и revenue, т.к. дальнейщий анализ проводим по revenue_rub

cadf.describe().drop(columns=['revenue', 'order_id', 'event_id'])

In [ ]:
def revenue_rub_bp_make(title):
    revenue_rub_bp = cadf.boxplot(
        column='revenue_rub', 
        figsize=(16,8), 
        vert=False)

    revenue_rub_bp.set_xscale('log')

    revenue_rub_bp.set_title(title)
    revenue_rub_bp.set_xlabel('Значение прибыли')

    return revenue_rub_bp

In [ ]:
revenue_rub_bp_make('Диаграмма размаха для числового параметра revenue_rub в логарифмическом масшатбе')

<b>Обнаружены выбросы в revenue_rub. Фильтруем данные по перцентилю 0.99</b>

In [ ]:
quantile_99 = cadf['revenue_rub'].quantile(0.99)

In [ ]:
number_of_excluded_spike_rows = cadf[cadf['revenue_rub'] >= quantile_99]['revenue_rub'].count()

In [ ]:
f'Количество строк с вревышением 99 перцентиля: {number_of_excluded_spike_rows}'

In [ ]:
cadf = cadf[cadf['revenue_rub'] < quantile_99].copy()

In [ ]:
revenue_rub_bp_make('Диаграмма размаха для числового параметра revenue_rub в логарифмическом масшатбе после удаления выбросов')

In [ ]:
tickets_count_bp = cadf.boxplot(
    column='tickets_count', 
    figsize=(16,8), 
    vert=False)

tickets_count_bp.set_title('Диаграмма размаха для числового параметра tickets_count')
tickets_count_bp.set_xlabel('Количесиво купленных билетов в одном заказе')

tickets_count_bp

In [ ]:
number_of_excluded_rows = number_of_excluded_spike_rows + number_of_negative_revenue_rows
f'Количество исключенных строк {number_of_excluded_rows}'

<b>
    <p>В результате проведенного анализа можно сделать вывод. </p>
    <p>В данных присутствуют строки с отраицательным значением прибыли, их 381, а также строки содержашие выбрсы по прибыли, их 3023. В сумме было исключено 3404 строки, что составляет около одного процента от всего датасета. Для выбросов были построены диаграммы размаха в логарифмическом масштабе до и после удаления значений прибыли, превышающих 99 перцентиль/</p>
    <p>Была проведена корректировка значения прибыли путем конверсии значений прибыли в kzt в rub. В результате была добавлена колонка revenue_rub содержащая в рублях.</p>
    <p>Также, был проведен анализ уникальных значений колонок с номинальными значениями, что позволило выявить, что в колонках  device_type_canonical и event_type_main присутствуют данные отражающие пропуски, а именно "other" и "другое" соответственно.</p>
</b>


---

### 3. Создание профиля пользователя

В будущем отдел маркетинга планирует создать модель для прогнозирования возврата пользователей. Поэтому сейчас они просят вас построить агрегированные признаки, описывающие поведение и профиль каждого пользователя.

---

**Задача 3.1.** Постройте профиль пользователя — для каждого пользователя найдите:

- дату первого и последнего заказа;
- устройство, с которого был сделан первый заказ;
- регион, в котором был сделан первый заказ;
- билетного партнёра, к которому обращались при первом заказе;
- жанр первого посещённого мероприятия (используйте поле `event_type_main`);
- общее количество заказов;
- средняя выручка с одного заказа в рублях;
- среднее количество билетов в заказе;
- среднее время между заказами.

После этого добавьте два бинарных признака:

- `is_two` — совершил ли пользователь 2 и более заказа;
- `is_five` — совершил ли пользователь 5 и более заказов.

**Рекомендация:** перед тем как строить профиль, отсортируйте данные по времени совершения заказа.

---


In [ ]:
profile_statistics_df = cadf.groupby('user_id').aggregate(
{
    'order_dt': ['min', 'max'],
    'revenue_rub': 'mean',
    'tickets_count': 'mean',
    'days_since_prev': 'mean',
    'tickets_count': 'mean',
    'order_id': 'count'
})

new_columns = []
for col, agg in profile_statistics_df.columns:
    new_columns.append(f"{col}_{agg}")

profile_statistics_df.columns = new_columns
profile_statistics_df = profile_statistics_df.reset_index()

profile_statistics_df['days_since_prev_mean'].fillna(0, inplace=True)

profile_statistics_df

In [ ]:
profile_first_df = cadf.sort_values('order_dt', ascending=True).drop_duplicates(subset='user_id', keep='first')[['user_id', 'device_type_canonical', 'region_name', 'service_name', 'event_type_main']]

profile_first_df

In [ ]:
profile_df = pd.merge(profile_statistics_df, profile_first_df, on='user_id', how='inner')

profile_df

In [ ]:
profile_df['is_two'] = profile_df['order_id_count'] >= 2

In [ ]:
profile_df['is_five'] = profile_df['order_id_count'] >= 5

In [ ]:
pdf = profile_df

In [ ]:
pdf

---

**Задача 3.2.** Прежде чем проводить исследовательский анализ данных и делать выводы, важно понять, с какими данными вы работаете: насколько они репрезентативны и нет ли в них аномалий.

Используя данные о профилях пользователей, рассчитайте:

- общее число пользователей в выборке;
- среднюю выручку с одного заказа;
- долю пользователей, совершивших 2 и более заказа;
- долю пользователей, совершивших 5 и более заказов.

Также изучите статистические показатели:

- по общему числу заказов;
- по среднему числу билетов в заказе;
- по среднему количеству дней между покупками.

По результатам оцените данные: достаточно ли их по объёму, есть ли аномальные значения в данных о количестве заказов и среднем количестве билетов?

Если вы найдёте аномальные значения, опишите их и примите обоснованное решение о том, как с ними поступить:

- Оставить и учитывать их при анализе?
- Отфильтровать данные по какому-то значению, например, по 95-му или 99-му перцентилю?

Если вы проведёте фильтрацию, то вычислите объём отфильтрованных данных и выведите статистические показатели по обновлённому датасету.

In [ ]:
total_users = pdf['user_id'].count()

In [ ]:
mean_order_revenue = (pdf['order_id_count'] * pdf['revenue_rub_mean']).sum() / pdf['order_id_count'].sum()

In [ ]:
more_than_two_users_share = pdf['is_two'].mean()

In [ ]:
more_than_five_users_share = pdf['is_five'].mean()

In [ ]:
print(f'Общее количество пользователей: {total_users}' )
print(f'Средняя выручка с одного заказа: {mean_order_revenue}' )
print(f'Долю пользователей, совершивших 2 и более заказа: {more_than_two_users_share}' )
print(f'Долю пользователей, совершивших 5 и более заказов: {more_than_five_users_share}' )

<b>Проведем анализ аномальных значений</b>

In [ ]:
def order_id_count_bp_make(title):

    order_id_count_bp = pdf.boxplot(
        column='order_id_count', 
        figsize=(16,8), 
        vert=False)

    order_id_count_bp.set_xscale('log')

    order_id_count_bp.set_title(title)
    order_id_count_bp.set_xlabel('Количесиво купленных билетов у пользователя')

    return order_id_count_bp

order_id_count_bp_make('Диаграмма размаха общего количества заказов order_id_count')

In [ ]:
def tickets_count_mean_bp_make(title):
    
    tickets_count_mean_bp = pdf.boxplot(
        column='tickets_count_mean', 
        figsize=(16,8), 
        vert=False)

    tickets_count_mean_bp.set_title(title)
    tickets_count_mean_bp.set_xlabel('Среднее количество билетов в заказе')

    return tickets_count_mean_bp

tickets_count_mean_bp_make('Диаграмма размаха среднего числа билетов в заказе tickets_count_mean')

In [ ]:
def days_since_prev_mean_bp_make(title):

    days_since_prev_mean_bp = pdf.boxplot(
        column='days_since_prev_mean', 
        figsize=(16,8), 
        vert=False)

    days_since_prev_mean_bp.set_title(title)
    days_since_prev_mean_bp.set_xlabel('Среднее число дней между заказами')

    return days_since_prev_mean_bp

days_since_prev_mean_bp_make('Диаграмма размаха среднего числа дней между заказами days_since_prev_mean')

<b>
    <p>Видно, что в колонках days_since_prev_mean и order_id_count присутствует большое количество аномалий. Выберем только значение меньше перцентиля 0.95 для order_id_count, т.к. разброс сильно выше, для days_since_prev_mean выберем 0.99</p>
    <p>В колонке tickets_count_mean иих количество незначительно, убирать их не будем</p>
</b>

In [ ]:
days_since_prev_mean_percentile_99 = pdf['days_since_prev_mean'].quantile(0.99)
days_since_prev_mean_percentile_99

In [ ]:
order_id_count_percentile_95 = pdf['order_id_count'].quantile(0.95)
order_id_count_percentile_95

In [ ]:
number_days_since_prev_mean_exceeds_percentile_99 = pdf[pdf['days_since_prev_mean'] > days_since_prev_mean_percentile_99]['days_since_prev_mean'].count()
number_order_id_count_exeeds_percentile_95 = pdf[pdf['order_id_count'] > order_id_count_percentile_95]['order_id_count'].count()

print('Количество пользователей, у которых значение среднего количества дней между заказами превышает 99 перцентиль:', number_days_since_prev_mean_exceeds_percentile_99)
print('Количество пользователей, у которых суммарное количество заказов превышает 95 перцентиль:', number_order_id_count_exeeds_percentile_95)
print('Всего удалено', number_days_since_prev_mean_exceeds_percentile_99 + number_order_id_count_exeeds_percentile_95)

In [ ]:
pdf = pdf[pdf['days_since_prev_mean'] < days_since_prev_mean_percentile_99].copy()
pdf = pdf[pdf['order_id_count'] < order_id_count_percentile_95].copy()

In [ ]:
order_id_count_bp_make('Диаграмма размаха общего количества заказов order_id_count после удаления')

In [ ]:
tickets_count_mean_bp_make('Диаграмма размаха среднего числа билетов в заказе tickets_count_mean после удаления')

In [ ]:
days_since_prev_mean_bp_make('Диаграмма размаха среднего числа дней между заказами days_since_prev_mean после удаления')

---

### 4. Исследовательский анализ данных

Следующий этап — исследование признаков, влияющих на возврат пользователей, то есть на совершение повторного заказа. Для этого используйте профили пользователей.



#### 4.1. Исследование признаков первого заказа и их связи с возвращением на платформу

Исследуйте признаки, описывающие первый заказ пользователя, и выясните, влияют ли они на вероятность возвращения пользователя.

---

**Задача 4.1.1.** Изучите распределение пользователей по признакам.

- Сгруппируйте пользователей:
    - по типу их первого мероприятия;
    - по типу устройства, с которого совершена первая покупка;
    - по региону проведения мероприятия из первого заказа;
    - по билетному оператору, продавшему билеты на первый заказ.
- Подсчитайте общее количество пользователей в каждом сегменте и их долю в разрезе каждого признака. Сегмент — это группа пользователей, объединённых определённым признаком, то есть объединённые принадлежностью к категории. Например, все клиенты, сделавшие первый заказ с мобильного телефона, — это сегмент.
- Ответьте на вопрос: равномерно ли распределены пользователи по сегментам или есть выраженные «точки входа» — сегменты с наибольшим числом пользователей?

---


In [ ]:
total_users_count = pdf['user_id'].count()

def group_by_sign_user_id_count(sign_name):
    df = pdf.groupby(sign_name)[['user_id']].count().reset_index().sort_values('user_id', ascending=False)
    
    df['share'] = df['user_id'] / total_users_count

    return df

def draw_bar_user_id(df, xlabel, title):
    df.plot.bar(
        x=list(df.columns)[0],
        y='user_id',
        figsize=(20,12),
        title=title
    )
    plt.xlabel(xlabel)
    plt.ylabel('Количество пользователей')
    plt.grid()
    plt.show()

In [ ]:
device_type_canonical_df = group_by_sign_user_id_count('device_type_canonical')
device_type_canonical_df

In [ ]:
draw_bar_user_id(device_type_canonical_df, 'Тип устройства', 'Распределение по типам устройств')

In [ ]:
event_type_main_df = group_by_sign_user_id_count('event_type_main')
event_type_main_df

In [ ]:
draw_bar_user_id(event_type_main_df, 'Тип события', 'Распределение по типам событий')

In [ ]:
region_name_df = group_by_sign_user_id_count('region_name')
region_name_df

In [ ]:
draw_bar_user_id(region_name_df.head(10), 'Регион', 'Распределение по регионам (топ 10)')

In [ ]:
service_name_df = group_by_sign_user_id_count('service_name')
service_name_df

In [ ]:
draw_bar_user_id(service_name_df.head(10), 'Сервис партнер', 'Распределение по сервисам партнерам (топ 10)')

<b>
    <p>Среди полученных данных можно заметить, что не наблюдается равномерное распределение</p>
    <p>Имеются явные точки входа пользователей. Не говорит о явной точке входа, скорей всего все остальные заказы далаются так же с мобильного телефона. Для проверки гипотезы нужно построить распределение по всем заказам, не только по первым.</p>
    <p>Большая часть билетов приобретается на концерты, театры и другое. Также не говорит о точках входа, это субъективные предпочтения. Либо говорит, но в принципе в большинстве своем на афише представлены концерты. Можно проверить посчитав сколько ивентов в каждой категории существует.</p>
    <p>Большинство заказов делается из Каменевский региона.</p>
    <p>Большинство заказов делается с платформ "Билеты без проблем", "Мой билет", "Лови билет", "Билеты в руки", "Облако". Это является основными точками входа, через котоыре приобретается большая часть билетов, более 70%</p>
</b>

---

**Задача 4.1.2.** Проанализируйте возвраты пользователей:

- Для каждого сегмента вычислите долю пользователей, совершивших два и более заказа.
- Визуализируйте результат подходящим графиком. Если сегментов слишком много, то поместите на график только 10 сегментов с наибольшим количеством пользователей. Такое возможно с сегментами по региону и по билетному оператору.
- Ответьте на вопросы:
    - Какие сегменты пользователей чаще возвращаются на Яндекс Афишу?
    - Наблюдаются ли успешные «точки входа» — такие сегменты, в которых пользователи чаще совершают повторный заказ, чем в среднем по выборке?

При интерпретации результатов учитывайте размер сегментов: если в сегменте мало пользователей (например, десятки), то доли могут быть нестабильными и недостоверными, то есть показывать широкую вариацию значений.

---


In [ ]:
def group_by_sign_is_two(sign_name):

    df = pdf.groupby(sign_name)[['is_two']].sum().reset_index()
    
    df['share'] = df['is_two'] / group_by_sign_user_id_count(sign_name)['user_id']

    return df.sort_values('share', ascending=False)

def draw_bar_is_two(df, xlabel, title):
    df.plot.bar(
        x=list(df.columns)[0],
        y='share',
        figsize=(20,12),
        title=title
    )
    plt.xlabel(xlabel)
    plt.ylabel('Доля пользователей с двумя иболее заказами')
    plt.grid()
    plt.show()

In [ ]:
device_type_canonical_df = group_by_sign_is_two('device_type_canonical')
device_type_canonical_df

In [ ]:
draw_bar_is_two(device_type_canonical_df, 'Тип устройства', 'Распределение по типам устройств')

In [ ]:
event_type_main_df = group_by_sign_is_two('event_type_main')
event_type_main_df

In [ ]:
draw_bar_is_two(event_type_main_df, 'Тип события', 'Распределение по типам событий')

In [ ]:
region_name_df = group_by_sign_is_two('region_name')
region_name_df = region_name_df[region_name_df['is_two'] > region_name_df['is_two'].quantile(0.25)]
region_name_df

In [ ]:
draw_bar_is_two(region_name_df.head(10), 'Регион', 'Распределение по регионам (топ 10)')

In [ ]:
service_name_df = group_by_sign_is_two('service_name')
service_name_df = service_name_df[service_name_df['is_two'] > service_name_df['is_two'].quantile(0.25)]
service_name_df

In [ ]:
draw_bar_is_two(service_name_df.head(10), 'Сервис партнер', 'Распределение по сервисам партнерам (топ 10)')

<b>
    <p>По всему датасету доля пользователей которая возвращается составляет около 0.62. Чаще всего, больше, чем в среднем возвращаются сегменты desktop, выставки, театр, Шантырский район, Святополянский округ, Край билетов, Дом культуры</p>
    <p>В целом по каждому сегменту доля пользователей совершивших 2 и более заказов отличается незначительно.</p>
</b>

---

**Задача 4.1.3.** Опираясь на выводы из задач выше, проверьте продуктовые гипотезы:

- **Гипотеза 1.** Тип мероприятия влияет на вероятность возврата на Яндекс Афишу: пользователи, которые совершили первый заказ на спортивные мероприятия, совершают повторный заказ чаще, чем пользователи, оформившие свой первый заказ на концерты.
- **Гипотеза 2.** В регионах, где больше всего пользователей посещают мероприятия, выше доля повторных заказов, чем в менее активных регионах.

---

<b>Гипотеза 1: По полученным данным можно заметить, что гипотеза не подтверждается. Доля возвратов пользователей, у которых первый заказ был по спортивному мероприятию меньше чем доля тех, у кого это был концерт. 0.533784 против 0.598422 соответственно</b>

<b>Гипотеза 2: Не подтверждается. Например, Каменевский регион. Количество посещающих в Каменевском регионе наибольшее, однако доля возвратов 0.6 далека от максимальной 0.88.</b>

---

#### 4.2. Исследование поведения пользователей через показатели выручки и состава заказа

Изучите количественные характеристики заказов пользователей, чтобы узнать среднюю выручку сервиса с заказа и количество билетов, которое пользователи обычно покупают.

Эти метрики важны не только для оценки выручки, но и для оценки вовлечённости пользователей. Возможно, пользователи с более крупными и дорогими заказами более заинтересованы в сервисе и поэтому чаще возвращаются.

---

**Задача 4.2.1.** Проследите связь между средней выручкой сервиса с заказа и повторными заказами.

- Постройте сравнительные гистограммы распределения средней выручки с билета (`avg_revenue_rub`):
    - для пользователей, совершивших один заказ;
    - для вернувшихся пользователей, совершивших 2 и более заказа.
- Ответьте на вопросы:
    - В каких диапазонах средней выручки концентрируются пользователи из каждой группы?
    - Есть ли различия между группами?

Текст на сером фоне:
    
**Рекомендация:**

1. Используйте одинаковые интервалы (`bins`) и прозрачность (`alpha`), чтобы визуально сопоставить распределения.
2. Задайте параметру `density` значение `True`, чтобы сравнивать форму распределений, даже если число пользователей в группах отличается.

---


In [ ]:
avg_ticket_df = pdf.copy()
avg_ticket_df['avg_revenue_rub'] = avg_ticket_df['revenue_rub_mean'] / avg_ticket_df['tickets_count_mean']
avg_ticket_df.head(10)

In [ ]:
def draw_compare_hist(df, df_condition_left, df_label_left, df_condition_right, df_label_right):

    plt.close()
    
    left = df[df_condition_left(df)][['avg_revenue_rub']].copy()
    left['source'] = df_label_left
    
    right = df[df_condition_right(df)][['avg_revenue_rub']].copy()
    right['source'] = df_label_right
    
    combined = pd.concat([left, right], ignore_index=True)
    
    plt.figure(figsize=(20, 12))
    
    hist = sns.histplot(
        data=combined,
        x='avg_revenue_rub',
        hue='source',
        stat='density',
        common_norm=False,
        alpha=0.5,
        palette='Set1',
        edgecolor='black',
        bins=200,
        legend=True,
        kde=True
    )
    
    left_mean = left['avg_revenue_rub'].mean()
    left_median = left['avg_revenue_rub'].median()

    hist.axvline(left_mean, color='blue', linestyle="--", linewidth=2, label="Mean")
    hist.axvline(left_median, color='blue', linestyle=":", linewidth=2, label="Median")

    right_mean = right['avg_revenue_rub'].mean()
    right_median = right['avg_revenue_rub'].median()

    hist.axvline(right_mean, color='red', linestyle="--", linewidth=2, label="Mean")
    hist.axvline(right_median, color='red', linestyle=":", linewidth=2, label="Median")

    plt.xlabel('Средняя выручка')
    plt.ylabel('Доля')
    plt.title('Гистограмма средней выручки')
    plt.grid(axis='y', alpha=0.3)
    plt.xlim(0, 700)


    plt.show()

In [ ]:
draw_compare_hist(avg_ticket_df, lambda df: df['is_two'] == False, 'Один заказ', lambda df: df['is_two'] == True, 'Два и более заказов')

<b>
    <p>Пользователи группируются в диапазоне средней выручки 200 рублей. Выручка с пользователей совершивших 2 и более заказов незначительно выше.</p>
    <p>Медианные значения различаются. У пользователей совершивших только один заказ наблюдается медиана с меньшим значением, чем у пользователей с двумя и более заказами</p>
</b>

---

**Задача 4.2.2.** Сравните распределение по средней выручке с заказа в двух группах пользователей:

- совершившие 2–4 заказа;
- совершившие 5 и более заказов.

Ответьте на вопрос: есть ли различия по значению средней выручки с заказа между пользователями этих двух групп?

---


In [ ]:
draw_compare_hist(avg_ticket_df, lambda df: ((df['is_two'] == True) & (df['is_five'] == False)), 'Два и более заказов', lambda df: df['is_five'] == True, 'Пять и более заказов')

In [ ]:
<b>
    <p>Среднее значение выручки между расматриваемыми группами не отличается, составляет также в районе 200 he,ktq.</p>
</b>

---

**Задача 4.2.3.** Проанализируйте влияние среднего количества билетов в заказе на вероятность повторной покупки.

- Изучите распределение пользователей по среднему количеству билетов в заказе (`avg_tickets_count`) и опишите основные наблюдения.
- Разделите пользователей на несколько сегментов по среднему количеству билетов в заказе:
    - от 1 до 2 билетов;
    - от 2 до 3 билетов;
    - от 3 до 5 билетов;
    - от 5 и более билетов.
- Для каждого сегмента подсчитайте общее число пользователей и долю пользователей, совершивших повторные заказы.
- Ответьте на вопросы:
    - Как распределены пользователи по сегментам — равномерно или сконцентрировано?
    - Есть ли сегменты с аномально высокой или низкой долей повторных покупок?

---

In [ ]:
bins = [0, 2, 3, 5, 1000]
labels = ['1-2 билета', '2-3 билета', '3-5 билетов', '5 и более билетов']

avg_tickets_count_df = pdf.copy()

avg_tickets_count_df['ticket_segment'] = pd.cut(pdf['tickets_count_mean'], bins=bins, labels=labels, right=True)

avg_tickets_count_df.groupby('ticket_segment').aggregate(
{
    'user_id': 'count',
    'is_two': 'mean'
})

<b>
    <p>Пользователи распределены неравномерною Больше всего повторных покупок в сегменте 2-3 билета (9014), меньше всего в 5 и более билетов (194).</p>
    <p>Сешментов с аномалиями не наблюдается.</p>
</b>

---

#### 4.3. Исследование временных характеристик первого заказа и их влияния на повторные покупки

Изучите временные параметры, связанные с первым заказом пользователей:

- день недели первой покупки;
- время с момента первой покупки — лайфтайм;
- средний интервал между покупками пользователей с повторными заказами.

---

**Задача 4.3.1.** Проанализируйте, как день недели, в которой была совершена первая покупка, влияет на поведение пользователей.

- По данным даты первого заказа выделите день недели.
- Для каждого дня недели подсчитайте общее число пользователей и долю пользователей, совершивших повторные заказы. Результаты визуализируйте.
- Ответьте на вопрос: влияет ли день недели, в которую совершена первая покупка, на вероятность возврата клиента?

---


In [ ]:
weekday_df = pdf.copy()

weekday_df['weekday'] = pdf['order_dt_min'].dt.weekday

agg_weekday_df = weekday_df.groupby('weekday').aggregate(
{
 'is_two': ['sum', 'mean']   
})

new_columns = []
for col, agg in agg_weekday_df.columns:
    new_columns.append(f"{col}_{agg}")

agg_weekday_df.columns = new_columns
agg_weekday_df = agg_weekday_df.reset_index()

weekday_names = {
    0: 'Понедельник',
    1: 'Вторник',
    2: 'Среда',
    3: 'Четверг',
    4: 'Пятница',
    5: 'Суббота',
    6: 'Воскресенье'
}

agg_weekday_df['weekday'] = agg_weekday_df.apply(lambda row : weekday_names[row['weekday']], axis=1)

agg_weekday_df

In [ ]:
def draw_bar_repeats(df, column_name, ylabel, title):
    df.plot.bar(
        x='weekday',
        y=column_name,
        figsize=(20,12),
        title=title
    )
    plt.xlabel('День недели')
    plt.ylabel(ylabel)
    plt.grid()
    plt.show()
    plt.close()
    

In [ ]:
draw_bar_repeats(agg_weekday_df[['weekday', 'is_two_sum']], 'is_two_sum', 'Количество повторных заказов', 'Диаграмма распределения количества повторных заказов по дням недели')

In [ ]:
draw_bar_repeats(agg_weekday_df[['weekday', 'is_two_mean']], 'is_two_mean', 'Доля повторных заказов', 'Диаграмма распределения долей повторных заказов по дням недели')

<b>
    <p>В целом зависимости от дня недели нет. Единственное, вероятность повторных заказов будет меньше, если первый заказ сделан в воскресенье.</p>
</b>

---

**Задача 4.3.2.** Изучите, как средний интервал между заказами влияет на удержание клиентов.

- Рассчитайте среднее время между заказами для двух групп пользователей:
    - совершившие 2–4 заказа;
    - совершившие 5 и более заказов.
- Исследуйте, как средний интервал между заказами влияет на вероятность повторного заказа, и сделайте выводы.

---


In [ ]:
pdf.columns

In [ ]:
days_since_prev_mean_df = pdf.copy()
days_since_prev_mean_df['days_since_prev'] = pdf['days_since_prev_mean'] * pdf['order_id_count']

days_since_prev_mean_24_df = days_since_prev_mean_df[(days_since_prev_mean_df['is_two'] == True) & days_since_prev_mean_df['is_five'] == False].copy()
days_since_prev_mean_5_df = days_since_prev_mean_df[(days_since_prev_mean_df['is_five'] == True)].copy()

days_since_prev_mean_24 = days_since_prev_mean_24_df['days_since_prev'].sum() / days_since_prev_mean_24_df['order_id_count'].sum()
days_since_prev_mean_5 = days_since_prev_mean_5_df['days_since_prev'].sum() / days_since_prev_mean_5_df['order_id_count'].sum()

print('Среднее время между заказами у пользователей совершивших 2-4 заказа', days_since_prev_mean_24)
print('Среднее время между заказами у пользователей совершивших 5 заказов', days_since_prev_mean_5)

<b>
    <p>Прослеживается зависимость, чем больше заказов совершает пользователь, тем меншь интервал между эти заказами, однакок, что то сказать про вероятность повторного заказа по параметру days_since_prev сложно. Все юзеры, у которых этот параметр указан уже совершили повторный заказ. Также, логично, что в среднем время снижается при увеличении количества заказов, т.к. исследуется фиксированный интервал времени.</p> 
<b>

---

#### 4.4. Корреляционный анализ количества покупок и признаков пользователя

Изучите, какие характеристики первого заказа и профиля пользователя могут быть связаны с числом покупок. Для этого используйте универсальный коэффициент корреляции `phi_k`, который позволяет анализировать как числовые, так и категориальные признаки.

---

**Задача 4.4.1:** Проведите корреляционный анализ:
- Рассчитайте коэффициент корреляции `phi_k` между признаками профиля пользователя и числом заказов (`total_orders`). При необходимости используйте параметр `interval_cols` для определения интервальных данных.
- Проанализируйте полученные результаты. Если полученные значения будут близки к нулю, проверьте разброс данных в `total_orders`. Такое возможно, когда в данных преобладает одно значение: в таком случае корреляционный анализ может показать отсутствие связей. Чтобы этого избежать, выделите сегменты пользователей по полю `total_orders`, а затем повторите корреляционный анализ. Выделите такие сегменты:
    - 1 заказ;
    - от 2 до 4 заказов;
    - от 5 и выше.
- Визуализируйте результат корреляции с помощью тепловой карты.
- Ответьте на вопрос: какие признаки наиболее связаны с количеством заказов?

---

In [ ]:
phik_df = pdf.copy()

phik_df.rename(columns={'order_id_count': 'total_orders'}, inplace=True)

phik_df['order_dt_min'] = phik_df['order_dt_min'].astype('int64')
phik_df['order_dt_max'] = phik_df['order_dt_max'].astype('int64')

phik_df = phik_df.drop(columns=['user_id'])

phik_df.columns

In [ ]:
def draw_heatmap(df):
    interval_cols = ['order_dt_min', 'order_dt_max', 'revenue_rub_mean', 'tickets_count_mean', 'days_since_prev_mean', 'total_orders']

    phik_matrix = df.phik_matrix(
        interval_cols=interval_cols
    )[['total_orders']]
    
    plt.figure(figsize=(10, 8))

    sns.heatmap(
        phik_matrix,
        annot=True,
        cmap='coolwarm',
        center=0,
        square=True,
        fmt='.2f',
        cbar_kws={'shrink': 0.8}
    )

    plt.title('Хитмэп взаимосвязей параметров', fontsize=16, pad=20)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
draw_heatmap(phik_df)

<b>
    <p>Четыре колоноки имеют околонулевые значения. Првоедем анализ по выделенным сегментам</p>
</b>

In [ ]:
phik_df_two = phik_df[(phik_df['is_two'] == True) & (phik_df['is_five'] == False)]

phik_df_two = phik_df_two.drop(columns=['is_two', 'is_five'])

draw_heatmap(phik_df_two)

In [ ]:
phik_df_two = phik_df[(phik_df['is_five'] == True)]

phik_df_two = phik_df_two.drop(columns=['is_two', 'is_five'])

draw_heatmap(phik_df_two)

<b>
    <p>Наибольшая зависимость total_orders прослеживается с order_dt_min, order_dt_max, days_since_prev_mean для сегмента 5 и более заказов. Есть незначительная корреляция с reegion_name 0.10. Для сегмента 2-4 заказа дополнительно прослеживается корреляция с ticket_count_mean.Также, заметна корреляция c revenue_rub_mean.</p>
</b>

### 5. Общий вывод и рекомендации

В конце проекта напишите общий вывод и рекомендации: расскажите заказчику, на что нужно обратить внимание. В выводах кратко укажите:

- **Информацию о данных**, с которыми вы работали, и то, как они были подготовлены: например, расскажите о фильтрации данных, переводе тенге в рубли, фильтрации выбросов.
- **Основные результаты анализа.** Например, укажите:
    - Сколько пользователей в выборке? Как распределены пользователи по числу заказов? Какие ещё статистические показатели вы подсчитали важным во время изучения данных?
    - Какие признаки первого заказа связаны с возвратом пользователей?
    - Как связаны средняя выручка и количество билетов в заказе с вероятностью повторных покупок?
    - Какие временные характеристики влияют на удержание (день недели, интервалы между покупками)?
    - Какие характеристики первого заказа и профиля пользователя могут быть связаны с числом покупок согласно результатам корреляционного анализа?
- Дополните выводы информацией, которая покажется вам важной и интересной. Следите за общим объёмом выводов — они должны быть компактными и ёмкими.

В конце предложите заказчику рекомендации о том, как именно действовать в его ситуации. Например, укажите, на какие сегменты пользователей стоит обратить внимание в первую очередь, а какие нуждаются в дополнительных маркетинговых усилиях.

   <b>В рамках проведенного исследования использовались данные базы данных Афиши. В ходе чистки были удалены 3404 строк по причинам аномальности данных (отрицательные значения прибыли, выбросы). Была проведена корректировка в рубли значения выручки на основании динамики курса тенге-рубль. Также была проведена корректировка типов данных в колонках на меньшую размерность. Данных с пропусками не обнаружено, они встречаются в days_since_prev, однако NaN имеет смысл. Это первый заказ пользователя.</b>

<b>Анализировались данные по 21753 пользователям. Распределены по количеству заказов следующим образом 1-2 билета 0.394285, 2-3 билета, 0.718993, 3-5 билетов, 0.614670, 5 и более билетов 0.329897.</b>

<b>В ходе исследования проводился анализ возвращаемости пользователя в зависимости от региона, типа события, сервиса партнера и типа устройства. Чаще всего возвращаются пользователи совершившие заказ с компьютера, из Озеропольской области, с сервиса Быстрый кассир, выставки </b>

<b>Количество покупок с двумя тремя билетами самое большое среди всех и составляет 9014. Также вероятность того, что пользователь совершит вторую покупку выше, если он берет больше билетов.</b>

<b>День недели не влияет на удержание клиента. Интервалы между покупками, дата начала первой, последней покупки сильно коррелируютс с количеством заказов.</b>

<b>Наибольшая зависимость total_orders прослеживается с order_dt_min, order_dt_max, days_since_prev_mean для сегмента 5 и более заказов. Есть незначительная корреляция с reegion_name 0.10. Для сегмента 2-4 заказа дополнительно прослеживается корреляция с ticket_count_mean.Также, заметна корреляция c revenue_rub_mean.</b>

<b>Общие рекоммендации</b>
<b>По данным заметно, что наибольшее количество первых заказов делается с мобильных устройств, но доля возвратившихся среди них ниже, чем у совершивших первый заказ с компьютера. Стоит уделять больше внимания именно мобильным клиентам, эффект от развития мобильных приложений будет выше, чем от развития десктопных.</b>

<b>Сервис "Билеты без проблем" хоть и составляет большинство первых заказов, однако доля возвратившихся с этого сервиса крайне низкая и не входит даже в топ 10. В своб очередь топ 3 сервиса по возвращению Быстрый кассир, Реестр? Crazy ticket! не входит в топ 10 партнеров по количеству билетов. Стоит попробовать увеличить количество билетов купленных через них, что приведет как к росту прибыли от пользователей совершающих первый заказ, так и повысит прибыль от их повторного заказа в дальнейшем.</b>

### 6. Финализация проекта и публикация в Git

Когда вы закончите анализировать данные, оформите проект, а затем опубликуйте его.

Выполните следующие действия:

1. Создайте файл `.gitignore`. Добавьте в него все временные и чувствительные файлы, которые не должны попасть в репозиторий.
2. Сформируйте файл `requirements.txt`. Зафиксируйте все библиотеки, которые вы использовали в проекте.
3. Вынести все чувствительные данные (параметры подключения к базе) в `.env`файл.
4. Проверьте, что проект запускается и воспроизводим.
5. Загрузите проект в публичный репозиторий — например, на GitHub. Убедитесь, что все нужные файлы находятся в репозитории, исключая те, что в `.gitignore`. Ссылка на репозиторий понадобится для отправки проекта на проверку. Вставьте её в шаблон проекта в тетрадке Jupyter Notebook перед отправкой проекта на ревью.

**Вставьте ссылку на проект в этой ячейке тетрадки перед отправкой проекта на ревью.**